In [ ]:
print("Good to GO!!!")

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [ ]:
extracted_data = load_pdf('Data/')

In [ ]:
print(extracted_data[15].page_content)

In [ ]:
def text_chunking(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

In [ ]:
text_chunks = text_chunking(extracted_data)

In [ ]:
text_chunks[0].page_content

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embedding_model = download_hugging_face_embeddings()

In [ ]:
embedding_model

In [ ]:
query_result = embedding_model.embed_query("Hello World!")

In [ ]:
query_result

In [ ]:
persist_directory='db'

In [ ]:
vector_store = Chroma.from_documents([text for text in text_chunks],embedding=embedding_model,persist_directory=persist_directory)

In [ ]:
retriever  = vector_store.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("Who was Harry's First Teacher?")

In [ ]:
docs

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer or there is no context provided , just say don't know , don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt = PromptTemplate(template=prompt_template,
                        input_variables=['context','question'])
chain_type_kwargs = {'prompt':prompt}

In [ ]:
from huggingface_hub import hf_hub_download

repo_id = "TheBloke/Llama-2-7B-Chat-GGML"
filename = "llama-2-7b-chat.ggmlv3.q4_0.bin"

file_path = hf_hub_download(repo_id=repo_id, filename=filename)

print(f"File downloaded to: {file_path}")

In [ ]:
llm = CTransformers(model=file_path,
                    model_type='llama',
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_store.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({'query':user_input})
    print("Response:",result['result'])